## Visualization of Peaks of a 4D image.

In this section we would be exploring how can we use the Horizon using python script.

Before starting let's make sure we have latest version of dipy and fury.

In [1]:
%pip install dipy --upgrade
%pip install fury --upgrade

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


Let's start by importing a few dipy modules and some widely used modules.

In [2]:
import os

from dipy.data import get_fnames
from dipy.data.fetcher import dipy_home
from dipy.io.image import load_nifti, load_nifti_data
from dipy.io.peaks import load_peaks, save_peaks
from dipy.io.gradients import read_bvals_bvecs
from dipy.core.gradients import gradient_table
from dipy.reconst.shm import CsaOdfModel
from dipy.direction import peaks_from_model
from dipy.data import get_fnames, default_sphere

from dipy.viz import horizon

We will be fetching some data in DIPY, but feel free to use your own data.

In [3]:
data_dir = 'stanford_hardi'
hardi_data = get_fnames(data_dir)

hardi_data

('/Users/maharshigor/.dipy/stanford_hardi/HARDI150.nii.gz',
 '/Users/maharshigor/.dipy/stanford_hardi/HARDI150.bval',
 '/Users/maharshigor/.dipy/stanford_hardi/HARDI150.bvec')

In [4]:
img = hardi_data[0]
bval = hardi_data[1]
bvec = hardi_data[2]

Let's try to read the fetched files

Usually this nifti files returns a lot of data from its shape, affine transformation, coordinate systems and other details.

Horizon accepts images either as a list of tuples.
Ex. `[(data, affine)]` or `[(data, affine, file_name)]`

Providing a `file_name` will also help you to identify the name of the file while visualzing it.

In [5]:
data, affine = load_nifti(img)
images=[(data, affine, img)]

In [6]:
label_fname = get_fnames('stanford_labels')
labels = load_nifti_data(label_fname)

white_matter = (labels == 1) | (labels == 2)

In [7]:
def get_whole_brain_peaks(
    img_data, img_affine, bval_fname, bvec_fname, mask=None):
    peaks_fname = os.path.join(dipy_home, data_dir, 'peaks.pam5')
    if os.path.isfile(peaks_fname):
        print(f'Loading: {peaks_fname}')
        peaks = load_peaks(peaks_fname, verbose=False)
    else:
        bvals, bvecs = read_bvals_bvecs(bval_fname, bvec_fname)
        gatb = gradient_table(bvals, bvecs=bvecs)
        sh_order = 8
        model = CsaOdfModel(gatb, sh_order)
        print('Calculating peaks...')
        peaks = peaks_from_model(
            model, img_data, default_sphere, relative_peak_threshold=.5, 
            min_separation_angle=25, mask=mask, sh_order=sh_order, normalize_peaks=True,
            parallel=False, num_processes=None)
        peaks.affine = img_affine
        print(f'Saving: {peaks_fname}')
        save_peaks(peaks_fname, peaks, affine=img_affine, verbose=True)
    return peaks

In [8]:
peaks = get_whole_brain_peaks(data, affine, bval, bvec, mask=white_matter)

Loading: /Users/maharshigor/.dipy/stanford_hardi/peaks.pam5


Once we read the files, the last step is to invoke the horizon with prepared data.

**Note for macOS users**

Please do not close the horizon window using close button if you are working with jupyter notebooks. Just stop the cell using stop the execution of the cell.

In [ ]:
horizon(images=images, pams=[peaks], world_coords=False)

/Users/maharshigor/miniforge3/envs/dipy-tests/lib/python3.9/site-packages/dipy/viz/horizon/util.py:61: UserWarning: int16 is not supported, falling back to int32
  warnings.warn(msg.format(data.dtype))


Original shape: (81, 106, 76, 160)
Resized to RAS shape: (81, 106, 76, 160)
